In [ ]:
import os
import sys
sys.path.append(os.path.abspath("..")) # set working dir

from web3 import Web3
from src.config import ALCHEMY_API_KEY

ETHEREUM_RPC_URL = f"https://eth-mainnet.g.alchemy.com/v2/{ALCHEMY_API_KEY}"
w3 = Web3(Web3.HTTPProvider(ETHEREUM_RPC_URL))
print(w3.is_connected()) # check connection
latest_block = w3.eth.get_block("latest")
print(latest_block) # latest block

In [ ]:
from src.config import OUTPUT_DIRECTORY
OUTPUT_DIRECTORY

# Setup

In [ ]:
from src.utils.retrieveAbi import save_abi_to_file
from src.config import MAINNET_UNISWAP_V2_ROUTER_02, MAINNET_SUSHISWAP_ROUTER_ADDRESS, CHAINID_MAINNET

# Load ABIs
save_abi_to_file(MAINNET_UNISWAP_V2_ROUTER_02) # uniswap v2 router abi
save_abi_to_file(MAINNET_SUSHISWAP_ROUTER_ADDRESS) # sushiswap router abi

# Contract Interactions

In [ ]:
import time
from src.utils.retrieveAbi import load_abi

def get_amounts_out(token1_address, token2_address, amount_in, router_contract):
    """Uses w3 getAmountsOut function to return amounts out, given two tokens, router contract and amount in"""
    token1_address = Web3.to_checksum_address(token1_address)
    token2_address = Web3.to_checksum_address(token2_address)
    path = [token1_address, token2_address]
    amounts_out = router_contract.functions.getAmountsOut(amount_in, path).call()
    return w3.from_wei(amounts_out[-1], 'ether')

In [ ]:
# uniswap
uniswap_router_abi = load_abi(MAINNET_UNISWAP_V2_ROUTER_02, CHAINID_MAINNET)
uniswap_router_address = Web3.to_checksum_address(MAINNET_UNISWAP_V2_ROUTER_02)
uniswap_router_contract = w3.eth.contract(address=uniswap_router_address, abi=uniswap_router_abi)
uniswap_weth_address = "0xC02aaa39b223FE8D0A0e5C4F27eAD9083C756Cc2"
uniswap_dai_address = "0x6B175474E89094C44Da98b954EedeAC495271d0F"
uniswap_amount_in = w3.to_wei(1, "ether")

# sushiswap
sushiswap_router_abi = load_abi(MAINNET_SUSHISWAP_ROUTER_ADDRESS, CHAINID_MAINNET)
sushiswap_router_address = Web3.to_checksum_address(MAINNET_SUSHISWAP_ROUTER_ADDRESS)
sushiswap_router_contract = w3.eth.contract(address=sushiswap_router_address, abi=sushiswap_router_abi)
sushiswap_weth_address = "0xC02aaa39b223FE8D0A0e5C4F27eAD9083C756Cc2"
sushiswap_dai_address = "0x6B175474E89094C44Da98b954EedeAC495271d0F"
sushiswap_amount_in = w3.to_wei(1, "ether")

try:
    prev_block=0
    while True:
        block = w3.eth.block_number
        uniswap_amount_out = get_amounts_out(uniswap_weth_address, uniswap_dai_address, uniswap_amount_in, uniswap_router_contract)
        sushiswap_amount_out = get_amounts_out(sushiswap_weth_address, sushiswap_dai_address, sushiswap_amount_in, sushiswap_router_contract)
        if block > prev_block:
            print(f"Block {block} | Uniswap: 1 ETH ≈ {uniswap_amount_out} DAI | Sushiswap: 1 ETH ≈ {sushiswap_amount_out} | Delta = {uniswap_amount_out-sushiswap_amount_out}")
            prev_block = block
        time.sleep(1)
except KeyboardInterrupt:
    print("Stopped by user.")

# Analysis

In [ ]:
import pandas as pd
from datetime import datetime, timezone

# Load the CSV
df_binance = pd.read_csv("/home/tobias/personal-dex-trading/out/data/order_book.csv", index_col="time")
df_binance = df_binance[~df_binance.index.duplicated(keep='first')]
df_binance

In [ ]:
# load csv
df_unichain = pd.read_csv("/home/tobias/personal-dex-trading/out/data/block_data.csv", index_col="timestamp")
df_unichain.index = pd.to_datetime(df_unichain.index, unit='s')
df_unichain.index += pd.Timedelta(hours=2) # match binance timezone
df_unichain

# Next

## Binance

Next, use SBE Market data for lower latency: https://developers.binance.com/docs/binance-spot-api-docs/sbe-market-data-streams

## Unichain

Next, we can either do:
- arbitrage between Uniswap v4 and Binance 


- arbitrage between Uniswap v2 and v4 pools